In [1]:
from pyfaidx import Fasta
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sfari= pd.read_csv("sfari_ed.csv")

genes = Fasta('Homo_sapiens.GRCh38.cdna.all.fa')


## Full list of the transcripts

In [2]:
names = pd.DataFrame(genes.keys(), columns=["id_version"])
names["id"]= names["id_version"].str.split(".").str.get(0)

In [3]:
names

,id_version,id
0,ENST00000415118.1,ENST00000415118
1,ENST00000448914.1,ENST00000448914
2,ENST00000434970.2,ENST00000434970
3,ENST00000631435.1,ENST00000631435
4,ENST00000632684.1,ENST00000632684
...,...,...
207244,ENST00000568156.1,ENST00000568156
207245,ENST00000567218.1,ENST00000567218
207246,ENST00000569984.1,ENST00000569984
207247,ENST00000624828.1,ENST00000624828


# Get the Canonical dataset

In [4]:
from pybiomart import Dataset

dataset = Dataset(name='hsapiens_gene_ensembl', host='http://www.ensembl.org')


df = dataset.query(attributes=['ensembl_gene_id', 'external_gene_name', 'ensembl_transcript_id', 'transcript_is_canonical'])

In [5]:
transcrip_df=df[df["Ensembl Canonical"]==1].copy() # filter to only show the canonical genes

In [6]:
transcrip_df

,Gene stable ID,Gene name,Transcript stable ID,Ensembl Canonical
0,ENSG00000210049,MT-TF,ENST00000387314,1.0
1,ENSG00000211459,MT-RNR1,ENST00000389680,1.0
2,ENSG00000210077,MT-TV,ENST00000387342,1.0
3,ENSG00000210082,MT-RNR2,ENST00000387347,1.0
4,ENSG00000209082,MT-TL1,ENST00000386347,1.0
...,...,...,...,...
276889,ENSG00000236500,CD24P1,ENST00000422383,1.0
276892,ENSG00000197312,DDI2,ENST00000480945,1.0
276898,ENSG00000215695,RSC1A1,ENST00000345034,1.0
276899,ENSG00000271742,NaN,ENST00000606262,1.0


### Merge the full transcipts with the cannonical transcripts

The purpose of this is to extract the Transcript version

In [7]:
cano_df = pd.merge(transcrip_df,names , left_on='Transcript stable ID', right_on='id')

In [8]:
clean_df = cano_df[["Gene name","Gene stable ID","id_version"]].copy()

In [9]:
clean_df

,Gene name,Gene stable ID,id_version
0,MT-ND1,ENSG00000198888,ENST00000361390.2
1,MT-ND2,ENSG00000198763,ENST00000361453.3
2,MT-CO1,ENSG00000198804,ENST00000361624.2
3,MT-CO2,ENSG00000198712,ENST00000361739.1
4,MT-ATP8,ENSG00000228253,ENST00000361851.1
...,...,...,...
41079,CHCHD2P6,ENSG00000235084,ENST00000454346.1
41080,CD24P1,ENSG00000236500,ENST00000422383.1
41081,DDI2,ENSG00000197312,ENST00000480945.6
41082,RSC1A1,ENSG00000215695,ENST00000345034.2


# Add the sequences

In [10]:
def get_seq(id):
    if id in genes:
                seq=genes[id][:].seq
                return seq

In [11]:
clean_dna=clean_df.copy()
clean_dna["seq"]=clean_dna['id_version'].apply(get_seq)

In [12]:
clean_dna

,Gene name,Gene stable ID,id_version,seq
0,MT-ND1,ENSG00000198888,ENST00000361390.2,ATACCCATGGCCAACCTCCTACTCCTCATTGTACCCATTCTAATCG...
1,MT-ND2,ENSG00000198763,ENST00000361453.3,ATTAATCCCCTGGCCCAACCCGTCATCTACTCTACCATCTTTGCAG...
2,MT-CO1,ENSG00000198804,ENST00000361624.2,ATGTTCGCCGACCGTTGACTATTCTCTACAAACCACAAAGACATTG...
3,MT-CO2,ENSG00000198712,ENST00000361739.1,ATGGCACATGCAGCGCAAGTAGGTCTACAAGACGCTACTTCCCCTA...
4,MT-ATP8,ENSG00000228253,ENST00000361851.1,ATGCCCCAACTAAATACTACCGTATGGCCCACCATAATTACCCCCA...
...,...,...,...,...
41079,CHCHD2P6,ENSG00000235084,ENST00000454346.1,GGAAGCCGAAGCCACACCTCCCGCATGGCCCCTCCGGCCAGCCGGG...
41080,CD24P1,ENSG00000236500,ENST00000422383.1,GCAATGGTGGACAGGCTCAGGCTGGGGCTGCTGCTTCTGGCACTGC...
41081,DDI2,ENSG00000197312,ENST00000480945.6,AGACGGACTCGCAGGCGTGTGGCGGCGGCCGTGCTTGCTAGTGAGG...
41082,RSC1A1,ENSG00000215695,ENST00000345034.2,AAGAGAAACCCGAGTTTGAGGACCTTATTTTATTCTACGCTGTTTA...


In [13]:
clean_dna.to_csv("transcript_seq.csv",index= False)

## Now to iterate the file

In [18]:
f= open("transcript_seq.csv","r")
print(f.readline())
print(f.readline().split(",")[3])
f.close()

Gene name,Gene stable ID,id_version,seq

ATACCCATGGCCAACCTCCTACTCCTCATTGTACCCATTCTAATCGCAATGGCATTCCTAATGCTTACCGAACGAAAAATTCTAGGCTATATACAACTACGCAAAGGCCCCAACGTTGTAGGCCCCTACGGGCTACTACAACCCTTCGCTGACGCCATAAAACTCTTCACCAAAGAGCCCCTAAAACCCGCCACATCTACCATCACCCTCTACATCACCGCCCCGACCTTAGCTCTCACCATCGCTCTTCTACTATGAACCCCCCTCCCCATACCCAACCCCCTGGTCAACCTCAACCTAGGCCTCCTATTTATTCTAGCCACCTCTAGCCTAGCCGTTTACTCAATCCTCTGATCAGGGTGAGCATCAAACTCAAACTACGCCCTGATCGGCGCACTGCGAGCAGTAGCCCAAACAATCTCATATGAAGTCACCCTAGCCATCATTCTACTATCAACATTACTAATAAGTGGCTCCTTTAACCTCTCCACCCTTATCACAACACAAGAACACCTCTGATTACTCCTGCCATCATGACCCTTGGCCATAATATGATTTATCTCCACACTAGCAGAGACCAACCGAACCCCCTTCGACCTTGCCGAAGGGGAGTCCGAACTAGTCTCAGGCTTCAACATCGAATACGCCGCAGGCCCCTTCGCCCTATTCTTCATAGCCGAATACACAAACATTATTATAATAAACACCCTCACCACTACAATCTTCCTAGGAACAACATATGACGCACTCTCCCCTGAACTCTACACAACATATTTTGTCACCAAGACCCTACTTCTAACCTCCCTGTTCTTATGAATTCGAACAGCATACCCCCGATTCCGCTACGACCAACTCATACACCTCCTATGAAAAAACTTCCTACCACTCACCCTAGCATTACTTATATGATATGTCTCCATACCCATTACAATCTCCAGCATTCCCCCTCAAACCTA



In [15]:
print(clean_dna.isnull().sum())

Gene name         6406
Gene stable ID       0
id_version           0
seq                  0
dtype: int64


In [1]:
import csv
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

Errors = []

with open("transcript_seq.csv", "r") as f:
    csv_reader = csv.reader(f)
    next(csv_reader)  # Skip the header row

    for row in csv_reader:
        if row[3] != "None":
            try:
                dna_seq = row[3]

                inputs = tokenizer(dna_seq, return_tensors='pt')["input_ids"]
                hidden_states = model(inputs)[0]
                # Embedding with mean pooling
                embedding_mean = torch.mean(hidden_states[0], dim=0)
                embedding = embedding_mean.detach().numpy()

                embedding_str = " ".join(map(str, embedding))
                row.append(embedding_str)

                # Open the output file for writing, write the modified row, and close the file
                with open("emb_file.csv", mode='a', newline='') as file:
                    csv_writer = csv.writer(file)
                    csv_writer.writerow(row)
            except:
                Errors.append(row[0])
                continue
f.close()
# cut of 10 000

c:\Users\joaop\anaconda3\envs\dna\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\joaop/.cache\huggingface\modules\transformers_modules\zhihan1996\DNABERT-2-117M\81ac6a98387cf94bc283553260f3fa6b88cef2fa\bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a m

In [ ]:
print("Errors: ", Errors)

In [8]:
import csv

# Specify the row number you want to check (1715)
row_number_to_check = 1715

with open("transcript_seq.csv", "r") as emb_file:
    
    csv_reader = csv.reader(emb_file)
    for index, row in enumerate(csv_reader):
        if index == row_number_to_check:
            print("Row 1715 in 'emb_file.csv' corresponds to:")
            print(row)
            break

Row 1715 in 'emb_file.csv' corresponds to:
['SMAD2', 'ENSG00000175387', 'ENST00000262160.11', '...']


34k seq size